In [7]:
import h3
import h3pandas
import numpy as np
import pandas as pd
import pydeck as pdk
import seaborn as sns
import networkx as nx
from cflp_function import *
from shapely.geometry import Polygon
from calculate_od import *
from pysal.lib import weights
from pysal.explore import esda
from splot.esda import moran_scatterplot, lisa_cluster, plot_local_autocorrelation

In [8]:
# import os
# os.environ['USE_PYGEOS'] = '0'
# import geopandas as gpd

In [69]:
hex = pd.read_csv('./hex/hex_df2.csv')
# get_fill_color(hex, "fuzzy", "viridis")

In [10]:
# Define a layer to display on a map
# layer = pdk.Layer(
#     "H3HexagonLayer",
#     hex,
#     pickable=True,
#     # stroked=True,
#     filled=True,
#     extruded=False,
#     auto_highlight=True,
#     opacity=0.6,
#     get_hexagon="hex9",
#     get_fill_color = 'color', 
#     get_line_color=[255, 255, 255],
#     line_width_min_pixels=0.1
#     )

# # Set the viewport location
# view_state = pdk.ViewState(longitude=6.747489560596507, latitude=52.316862707395394, zoom=9, bearing=0, pitch=0)

# # Render
# r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "Value: {fuzzy}"})
# r.to_html("layer.html")

### H3 indices and polygon geometries
We want to have a geodataframe that stores the H3 indices in study area and the polygon geometries

In [11]:
# lst = gpd.read_file('./h3_list.shp')

In [12]:
# idx = pd.DataFrame(index=lst.hex9)

In [13]:
# idx = idx.h3.h3_to_geo_boundary()

In [14]:
# idx = gpd.GeoDataFrame(idx, geometry='geometry', crs='EPSG:4326')

In [15]:
# idx.to_file('./h3_polygons.shp')

In [16]:
# def cell_to_shapely_polygon(h3_index):
#     """
#     A function to convert H3 index to Shapely polygons

#     """
#     # hex_center_coords = h3.h3_to_geo(h3_index)
#     coords = h3.h3_to_geo_boundary(h3_index)
#     flipped = tuple(coord[::-1] for coord in coords)
#     # center_point = Point(hex_center_coords)
#     return Polygon(flipped) #, center_point

In [17]:
# hex['geometry'] = hex['hex9'].apply(cell_to_shapely_polygon) # can change the function here
# gdf = gpd.GeoDataFrame(hex, geometry='geometry', crs=4326)

In [18]:
idx = gpd.read_file('./h3_polygons.shp')

In [19]:
idx = idx.set_index('hex9')

In [20]:
df = pd.DataFrame(index=idx.index)
df.head()

""
hex9
891f1655aa3ffff
891f1655aa7ffff
891f1655bd3ffff
891f1655bd7ffff
891f1655b8bffff


### Merge compound suitability map and H3 geometry information

In [21]:
# gdf = idx.merge(hex[['hex9','color','fuzzy']], on='hex9',how='left')

### Spatial Autocorrelation

In [22]:
w = weights.Queen.from_dataframe(idx, use_index=True)

c:\Users\User\anaconda3\envs\scip\lib\site-packages\libpysal\weights\weights.py:224: UserWarning: The weights matrix is not fully connected: 
 There are 8 disconnected components.
 There are 2 islands with ids: 891f16c693bffff, 891f1610e8fffff.
  warnings.warn(message)


In [23]:
# w_queen.min_neighbors

In [24]:
# w_queen.max_neighbors

In [25]:
# w_queen.islands

In [26]:
# # view islands - those that don't have any neighbors
# ax=idx.plot(color='k', figsize=(6,6))
# idx.loc[w_queen.islands,:].plot(color='red', ax=ax)

In [27]:
# Drop the islands
# idx = idx.drop(w_queen.islands)

In [28]:
# recalculate spatial weights matrix
# w = weights.Queen.from_dataframe(idx, use_index=True)

In [29]:
# Row standardize the matrix
w.transform = 'R'

('WARNING: ', '891f16c693bffff', ' is an island (no neighbors)')
('WARNING: ', '891f1610e8fffff', ' is an island (no neighbors)')


In [30]:
# Reindex the compound suitability df such that its indices are ordered as the H3 list geodataframe
# hex_df = hex.set_index('hex9').reindex(idx.index)
hex_df = idx.merge(hex[['hex9','color','fuzzy']], on='hex9',how='left')

In [31]:
# Get standardised fuzzy values
hex_df['fuzzy_std'] = (hex_df['fuzzy'] - hex_df['fuzzy'].mean()) / hex_df['fuzzy'].std()

In [32]:
hex_df['lag_std'] = weights.lag_spatial(w, hex_df['fuzzy_std'])

In [33]:
hex_df['lag'] = weights.lag_spatial(w, hex_df['fuzzy'])

In [34]:
# # Plot global moran
# f, ax = plt.subplots(1, figsize=(5, 5))
# # Plot values
# sns.regplot(x='fuzzy', y='lag', data=hex_df, ci=None)
# # Add vertical and horizontal lines
# # plt.axvline(0, c='k', alpha=0.5)
# # plt.axhline(0, c='k', alpha=0.5)
# # # plt.text(1.75, 0.5, "HH", fontsize=25)
# # # plt.text(1.5, -1.5, "HL", fontsize=25)
# # # plt.text(-2, 1, "LH", fontsize=25)
# # # plt.text(-1.5, -2.5, "LL", fontsize=25)
# # Display
# plt.show()

In [35]:
# # Plot standardized moran's i
# f, ax = plt.subplots(1, figsize=(5, 5))
# # Plot values
# sns.regplot(x='fuzzy_std', y='lag_std', data=hex_df, ci=None, line_kws=dict(color="r"))
# # Add vertical and horizontal lines
# plt.axvline(0, c='k', alpha=0.5)
# ax.set_xlabel('Standarized Suitability Values')
# plt.axhline(0, c='k', alpha=0.5)
# ax.set_ylabel('Spatial Lag')
# plt.text(0.5, 2, "HH", fontsize=25)
# plt.text(0.5, -2.5, "HL", fontsize=25)
# plt.text(-5, 2, "LH", fontsize=25)
# plt.text(-5, -2.5, "LL", fontsize=25)
# # Display
# plt.show()

**Local Moran's I** tells you whether the local spatial pattern around each observation is similar to the overall pattern in the entire dataset.
**p_sim** is simulated p-values, which represent the probability of observing a similar or more extreme local spatial pattern under the assumption of spatial randomness.


In [36]:
lisa = esda.Moran_Local(hex_df['fuzzy'], w, seed=42)
# Break observations into significant or not
hex_df['significant'] = lisa.p_sim < 0.05
# Store the quadrant they belong to
hex_df['quadrant'] = lisa.q

In [37]:
hex_df['significant_0.01'] = lisa.p_sim < 0.01

In [38]:
np.sum(lisa.p_sim < 0.05)

4457

In [39]:
np.sum(lisa.p_sim < 0.01)

2540

In [40]:
# lisa_cluster(lisa, hex_df)

In [41]:
# plot_local_autocorrelation(lisa, hex_df, 'fuzzy')

In [42]:
# ax = hex_df.plot(color='k', figsize=(9, 9))
# hex_df[(hex_df['significant'] == True) & (hex_df['quadrant'] == 1)].plot(color='red', ax=ax)
# ax.set_axis_off()

### HH Clusters to Connected Graphs

In [43]:
# pd.DataFrame(*w.full()).astype(int)

In [44]:
# Transform weight back to Binary
w.transform = 'B'

In [45]:
g = w.to_networkx()

In [46]:
print("Number of nodes:", nx.number_of_nodes(g))
print("Number of edges:", nx.number_of_edges(g))

Number of nodes: 11628
Number of edges: 31726


In [47]:
# NetworkX relabeled the nodes with integer starting from 0, we want the nodes to be labeled by their H3 index 
mapping = {new_index: old_index for new_index, old_index in enumerate(w.id_order)}

In [48]:
# Apply the mapping to the graph
g = nx.relabel_nodes(g, mapping)

In [49]:
# index_to_polygon = dict(zip(hex_df['hex9'], hex_df['geometry']))

In [50]:
# nx.set_node_attributes(g, index_to_polygon, 'coordinates')

In [51]:
# Get indices of H3 cells that are in the HH quadrant (p-value < 0.05)
# HH = hex_df[(hex_df['significant'] == True) & (hex_df['quadrant'] == 1)].hex9.to_list()

In [52]:
# Get indices of H3 cells that are in the HH quadrant (p-value < 0.01)
hh = hex_df[(hex_df['significant_0.01'] == True) & (hex_df['quadrant'] == 1)].hex9.to_list()

In [53]:
print(len(hh))

1337


In [54]:
# Get sub graph that includes only the HH quadrant
# H = g.subgraph(HH)

In [55]:
h = g.subgraph(hh)

In [56]:
# print("Number of nodes p=0.05:", nx.number_of_nodes(H))
print("Number of nodes p=0.01:", nx.number_of_nodes(h))

Number of nodes p=0.01: 1337


In [57]:
# nx.write_graphml(g, './g.graphml')

In [58]:
# nx.write_graphml(H, './H.graphml')

In [59]:
# subH = list(nx.connected_components(H))
subh = list(nx.connected_components(h))
print(len(subh))

70


In [60]:
# Get connected graphs without islands (connected graphs consist of a single node)
# filter_subH = [component for component in subH if len(component) > 10]
filter_subh = [component for component in subh if len(component) > 10]
print('Number of connected grph in HH sub graph', len(filter_subh))

Number of connected grph in HH sub graph 23


In [61]:
# Calculate eigenvector centrality for each connected component
eigenvector_centralities = []

for component in filter_subh:
    # Create a subgraph for the current connected component
    subgraph = h.subgraph(component)
    
    # Calculate eigenvector centrality for the subgraph
    eigenvector_centrality = nx.eigenvector_centrality(subgraph, max_iter=1000)
    
    # Append the result to the list
    eigenvector_centralities.append(eigenvector_centrality)

In [62]:
# # Print or use the list of eigenvector centralities as needed
# for i, centrality in enumerate(eigenvector_centralities):
#     print(f"Connected Component {i + 1}: {centrality}")

In [63]:
# export for visualization in ArcGIS
eigen = hex_df[['hex9','geometry']]

In [64]:
eigen['eigen_centrality'] = np.nan

c:\Users\User\anaconda3\envs\scip\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [65]:
# Fill in eigenvector centralities into the 'eigen' column based on 'hex9'
for centrality_dict in eigenvector_centralities:
    for node_index, centrality_value in centrality_dict.items():
        # Update the 'eigen' column with the corresponding centrality value
        eigen.loc[eigen['hex9'] == node_index, 'eigen_centrality'] = centrality_value

In [66]:
# eigen[~eigen['eigen_centrality'].isna()]

In [67]:
site = []
for centrality_dict in eigenvector_centralities:
    max_node_index = max(centrality_dict, key=centrality_dict.get)
    site.append(max_node_index)

In [ ]:
# eigen[~eigen['eigen_centrality'].isna()].to_file('./eigen.shp')

In [ ]:
# sub_size = [len(c) for c in sorted(nx.connected_components(H), key=len, reverse=True)]
# sub_sizeh = [len(c) for c in sorted(nx.connected_components(h), key=len, reverse=True)]

In [ ]:
# # Plotting the histogram
# plt.hist(sub_size, bins=30, edgecolor='black')
# plt.xlabel('Connected Component Size')
# plt.ylabel('Frequency')
# plt.title('Distribution of Connected Component Sizes')
# plt.show()

In [ ]:
# hex_df[hex_df['hex9'].isin(nH)].to_file('./n005.shp')

In [ ]:
# hex_df[hex_df['hex9'].isin(list(h.nodes()))].to_file('./n001.shp')

In [ ]:
# Counter(sub_size)

In [ ]:
# Counter(sub_sizeh)

In [ ]:
# from collections import Counter
# # Use Counter to count occurrences of each value
# value_counts = Counter(sub_size)

# # Get the 10 most common values
# most_common_values = value_counts.most_common(10)

# # Display the result(f"{value}: {count}")
# print("10 most common values and their counts:")
# for value, count in most_common_values:
#     print

In [ ]:
hex_df = hex.set_index('hex9').reindex(idx.index)

In [90]:
hex_df

,hex9,geometry,color,fuzzy,fuzzy_std,lag_std,lag,significant,quadrant,significant_0.01
0,891f1655aa3ffff,"POLYGON ((6.72147 52.11887, 6.72374 52.11996, ...","[249, 121, 92]",0.720,0.948215,0.687780,0.698333,False,1,False
1,891f1655aa7ffff,"POLYGON ((6.72629 52.11945, 6.72857 52.12054, ...","[246, 112, 91]",0.703,0.743874,0.894125,0.715500,True,1,False
2,891f1655bd3ffff,"POLYGON ((6.73112 52.12003, 6.73339 52.12112, ...","[249, 123, 93]",0.726,1.020336,0.800969,0.707750,False,1,False
3,891f1655bd7ffff,"POLYGON ((6.73595 52.12060, 6.73822 52.12170, ...","[249, 125, 93]",0.727,1.032356,0.719833,0.701000,False,1,False
4,891f1655b8bffff,"POLYGON ((6.74077 52.12118, 6.74304 52.12227, ...","[246, 112, 91]",0.700,0.707813,0.734859,0.702250,False,1,False
...,...,...,...,...,...,...,...,...,...,...
11623,891f16d5cd3ffff,"POLYGON ((6.51051 52.48888, 6.51279 52.48997, ...","[200, 61, 114]",0.545,-1.155303,-0.749023,0.578800,False,3,False
11624,891f16d5cd7ffff,"POLYGON ((6.51535 52.48947, 6.51763 52.49056, ...","[194, 58, 117]",0.530,-1.335604,-0.419071,0.606250,False,3,False
11625,891f16d5c8bffff,"POLYGON ((6.52020 52.49006, 6.52248 52.49115, ...","[228, 78, 100]",0.618,-0.277835,-0.422076,0.606000,False,3,False
11626,891f16d51b7ffff,"POLYGON ((6.50281 52.49039, 6.50509 52.49149, ...","[222, 74, 103]",0.605,-0.434096,-0.361976,0.611000,False,3,False


In [83]:
def get_sites(fuzzy_df, w, g, idx):
    fuzzy_df = fuzzy_df.set_index('hex9').reindex(idx.index)
    # 1. Compute loca moran's I
    lisa = esda.Moran_Local(fuzzy_df['fuzzy'], w, seed=42)
    # 2. Break observations into significant or not
    # fuzzy_df['significant'] = lisa.p_sim < 0.01
    # # 3. Store the quadrant they belong to
    # fuzzy_df['quadrant'] = lisa.q
    # # Get indices of H3 cells that are in the HH quadrant
    # HH = fuzzy_df[(fuzzy_df['significant'] == True) & (fuzzy_df['quadrant'] == 1)].hex9.to_list()
    HH = fuzzy_df[(lisa.q == 1) & (lisa.p_sim < 0.01)].index.to_list()
    # Build sub graph that includes only the HH quadrant
    H = g.subgraph(HH)
    # Get sub components in the sub graphs
    subH = list(nx.connected_components(H))
    filter_subH = [component for component in subH if len(component) > 10]
    len(filter_subH)
    # Calculate eigenvector centrality for each connected component
    site_idx = []
    for component in filter_subH:
        # Create a subgraph for the current connected component
        subgraph = H.subgraph(component)
        # Calculate eigenvector centrality for a connected graph
        eigenvector_centrality = nx.eigenvector_centrality(subgraph, max_iter=1000)
        # Get the node index with the highest eigenvector centrality in that connected graph
        max_node_index = max(eigenvector_centrality, key=eigenvector_centrality.get)
        # Append the node index to a list
        site_idx.append(max_node_index)
    return site_idx

In [85]:
site_idx = get_sites(hex, w, g, idx)

In [111]:
hex = hex.set_index('hex9')

In [114]:
len(hex.loc[site_idx].reset_index())

23

In [86]:
print(len(site), len(site_idx))
set(site) & set(site_idx)

23 23


{'891f160a87bffff',
 '891f160aecfffff',
 '891f160b4dbffff',
 '891f16106c7ffff',
 '891f1610e63ffff',
 '891f1611437ffff',
 '891f16190b3ffff',
 '891f161940bffff',
 '891f1619933ffff',
 '891f161b593ffff',
 '891f1646097ffff',
 '891f1652ecfffff',
 '891f16541b7ffff',
 '891f1655cc3ffff',
 '891f165660bffff',
 '891f165689bffff',
 '891f1657497ffff',
 '891f16c0b13ffff',
 '891f16c6c2bffff',
 '891f16c894fffff',
 '891f16cc20bffff',
 '891f16cca87ffff',
 '891f16cd443ffff'}

In [ ]:
hex_df[hex_df['hex9'].isin(site_idx)].to_file('./site_idx.shp')

In [ ]:
hex_df[hex_df['hex9'].isin(site)].to_file('./site.shp')

In [87]:
# Define a layer to display on a map
layer = pdk.Layer(
        "H3HexagonLayer",
        pd.DataFrame({'hex9': site_idx}),
        pickable=True,
        stroked=True,
        filled=True,
        extruded=False,
        opacity=0.6,
        get_hexagon="hex9",
        get_fill_color=[0, 255, 0, 0], 
        get_line_color=[0, 255, 0],
        line_width_min_pixels=2)
layer2 = pdk.Layer(
        "H3HexagonLayer",
        pd.DataFrame({'hex9': site}),
        pickable=True,
        stroked=True,
        filled=True,
        extruded=False,
        opacity=0.6,
        get_hexagon="hex9",
        get_fill_color=[0, 0, 0, 0], 
        get_line_color=[255, 0, 0],
        line_width_min_pixels=2)

# Set the viewport location
view_state = pdk.ViewState(longitude=6.747489560596507, latitude=52.316862707395394, zoom=9, bearing=0, pitch=0)

# Render
r = pdk.Deck(layers=[layer, layer2], initial_view_state=view_state, tooltip={"text": "Count: {value}"})
r.to_html("h3_hexagon_layer_cropped_new.html")

In [ ]:
import gc
gc.collect()